In [ ]:
from pysr import PySRRegressor
#import simpy as sp
import os
import yaml
import numpy as np

import json


#PATH = '/home/jpierre/v2/pySr/data/mod1/data.json'
#PATH_CFG = '/master/code/experimental/pysr/cfg_pysr'
PATH = '/home/jpierre/v2/pySr/data/normal/normal_action-None_dropout0_l1-0-lim0.json'

NB_RUN = 1000
BINARY_OP = ["+", "*", 'cond']
UNARY_OP = ["inv(x) = 1/x"]
VARIABLES = ['v_x_0', 'v_y_0', 'v_x_1', 'v_y_1', 'v_x_2', 'v_y_2', 'v_x_3', 'v_y_3', 'd', 'delta_x', 'delta_y', 'r_i', 'r_j']
MAX_SIZE = 13
PARSIMONY = 0.0000032



def readJson(filePath:str):
    """
    Function to read json 
    """
    
    with open(filePath, 'r') as f:
        data = json.load(f)
    return data


def writeJson(data, filePath):
    """
    Function to write json 
    """
    with open(filePath, 'w') as f:
        json.dump(data, f, indent=2)


def getData(jsonPath = PATH):
    # for now


    messages = np.array(readJson(jsonPath)['messages'])
    edges = np.array(readJson(jsonPath)['edges'])[:, :3]

    return edges, messages



def getPySrModel(nbRun = NB_RUN, binaryOp = BINARY_OP, unaryOp = UNARY_OP, maxsize = MAX_SIZE):

    model = PySRRegressor(
        niterations=nbRun,
        binary_operators=binaryOp,
        #unary_operators= ["inv(x) = 1/x",],
        #populations=15,
        model_selection = "best",
        maxsize = maxsize,
        #complexity_of_variables = 2,
        #parsimony = PARSIMONY,
        #nested_constraints = {"cond": {"*":0}},
        #adaptive_parsimony_scaling = 1000,
        #ncycles_per_iteration = 1000,
        #turbo = True,
        #extra_sympy_mappings = {"inv": lambda x: 1 / x}
        elementwise_loss = 'L1DistLoss()',
        weight_optimize = 0.001
    )

    return model


def fittingModel(model, X, y, verbose:bool = False, variables = VARIABLES):

    if verbose:
        print(">>>>> Fitting pySr")


    # Fit model
    model.fit(X, y, 
    variable_names = variables,
    # X_units = [""],
    # y_units = "",
    complexity_of_variables = [1, 3, 3]
    )


    return model

In [ ]:
X, Y = getData()

In [ ]:
print(X.shape)
print(Y.shape)

print(np.min(X[:, 0]))
print(np.max(X[:, 0]))

In [ ]:
pyReg = getPySrModel()

mod = fittingModel(pyReg, X, Y, variables=[' r ', ' cosine ', ' sine '])

In [ ]:
print(mod.latex_table(indices=None, precision=3, columns=['equation', 'complexity', 'loss', 'score']))

In [ ]:
print(mod.sympy())